# Predictions

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.feature_selection import chi2, f_regression, f_classif, mutual_info_classif, mutual_info_regression 
from sklearn.feature_selection import SelectKBest, SelectFromModel, VarianceThreshold
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
from sklearn import neighbors
from sklearn import ensemble
from sklearn import model_selection as ms
sklearn.set_config(print_changed_only=False)

from matplotlib import pyplot as plt

from ipynb.fs.full.Self_Written_Functions_Sheet import col_obj_list, nan_col_list, cols_nan_unique, nan_col_obj_list_fillna, fill_masvnrtype, conversion

# Basic EDA and Imputations

In [3]:
# Import original dataset to compare NaNs
hp_w_outcome = pd.read_csv('../train.csv')

# Import prepared dataset to cross-check validity of cleaned test dataset
dataset = pd.read_csv('../hp_for_dummies.csv')

# Import test dataset
hp_test_csv = pd.read_csv('../test.csv')

hp_test_csv.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [4]:
hp_w_outcome = hp_w_outcome.drop(['SalePrice'], axis=1)


ds_cols = [x for x in hp_w_outcome.columns]
hp_na = [x for x in hp_test_csv.columns if hp_test_csv[x].isna().any()]

hp_na

['MSZoning',
 'LotFrontage',
 'Alley',
 'Utilities',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType']

In [5]:
# columns that have dtype=='object' vs not
cols_obj = []
cols_num = []

# columns with object dtype that have null values
cols_w_nan_obj = []

# columns with non-object dtype that have null values
cols_w_nan_num = []

# Creates list of column names with dtype=='object' and one where dtype!='object'
col_obj_list(hp_test_csv, hp_test_csv.columns, cols_obj, cols_num)

# Outputs a list of the columns that have null values
nan_col_list(hp_test_csv, cols_obj, cols_w_nan_obj)

# Outputs a list of the columns that have null values
nan_col_list(hp_test_csv, cols_num, cols_w_nan_num)

data_dict_obj = {}
data_dict_null_obj = {}
cols_nan_unique(hp_test_csv, cols_w_nan_obj, data_dict_obj, data_dict_null_obj)

# to visualize via dataframes the unique values that exist in the dataset
# to visualize the null count next to those values

data_dict_obj_df = pd.DataFrame.from_dict(data_dict_obj, orient='index').fillna('')
data_dict_null_obj_df = pd.DataFrame(data_dict_null_obj.items())
data_dict_null_obj_df.columns = ['Col_Name', 'Null_Count']
data_dict_null_obj_df = data_dict_null_obj_df.set_index('Col_Name')

pd.concat([data_dict_obj_df, data_dict_null_obj_df], axis=1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,Null_Count
MSZoning,RH,RL,RM,FV,C (all),,,,,,,,,,,4
Alley,Pave,Grvl,,,,,,,,,,,,,,1352
Utilities,AllPub,,,,,,,,,,,,,,,2
Exterior1st,VinylSd,Wd Sdng,HdBoard,Plywood,MetalSd,CemntBd,WdShing,BrkFace,AsbShng,BrkComm,Stucco,AsphShn,CBlock,,,1
Exterior2nd,VinylSd,Wd Sdng,HdBoard,Plywood,MetalSd,Brk Cmn,CmentBd,ImStucc,Wd Shng,AsbShng,Stucco,CBlock,BrkFace,AsphShn,Stone,1
MasVnrType,None,BrkFace,Stone,BrkCmn,,,,,,,,,,,,16
BsmtQual,TA,Gd,Ex,Fa,,,,,,,,,,,,44
BsmtCond,TA,Po,Fa,Gd,,,,,,,,,,,,45
BsmtExposure,No,Gd,Mn,Av,,,,,,,,,,,,44
BsmtFinType1,Rec,ALQ,GLQ,Unf,BLQ,LwQ,,,,,,,,,,42


In [6]:
hp_test_csv['Electrical'].fillna(hp_test_csv['Electrical'].mode()[0], inplace=True)
hp_test_csv[hp_test_csv['Electrical'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


In [7]:
data_dict_num = {}
data_dict_null_num = {}
cols_nan_unique(hp_test_csv, cols_w_nan_num, data_dict_num, data_dict_null_num)

# data_dict_obj_df = pd.DataFrame.from_dict(data_dict_obj, orient='index').fillna('')
data_dict_null_num_df = pd.DataFrame(data_dict_null_num.items())
data_dict_null_num_df.columns = ['Col_Name', 'Null_Count']
data_dict_null_num_df = data_dict_null_num_df.set_index('Col_Name')
data_dict_null_num_df

,Null_Count
Col_Name,
LotFrontage,227
MasVnrArea,15
BsmtFinSF1,1
BsmtFinSF2,1
BsmtUnfSF,1
TotalBsmtSF,1
BsmtFullBath,2
BsmtHalfBath,2
GarageYrBlt,78


In [8]:
hp_test_csv.loc[hp_test_csv.MasVnrArea.isna(), 'MasVnrArea'] = 0
hp_test_csv[hp_test_csv['MasVnrArea'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


In [9]:
hp_test_csv.loc[(hp_test_csv.MasVnrType.isna()) & (hp_test_csv.MasVnrArea==0), 'MasVnrType']='None'
hp_test_csv.loc[(hp_test_csv.MasVnrType.isna()) & (hp_test_csv.MasVnrArea!=0), 'MasVnrType']='None'

In [10]:
hp_test_csv[(hp_test_csv['MasVnrType']=='None') & (hp_test_csv['MasVnrArea']!=0)][['MasVnrType', 'MasVnrArea']]

,MasVnrType,MasVnrArea
209,None,285.0
992,None,1.0
1150,None,198.0


In [11]:
hp_test_csv.loc[hp_test_csv.MasVnrArea==1, 'MasVnrArea'] = 0

In [12]:
# ['MasVnrType'] should = the mode of the ['MasVnrType'] that is NOT None

from collections import Counter

masvnrtype_mode = [x[0] for x in Counter(list(hp_test_csv['MasVnrType'])).most_common()]
    # outputs something that looks like: ['None', 'BrkFace', 'Stone', 'BrkCmn']

if masvnrtype_mode[0]=='None':
    hp_test_csv.loc[(hp_test_csv.MasVnrArea > 1) & (hp_test_csv.MasVnrType == 'None'), 'MasVnrType'] = masvnrtype_mode[1]
else:
    hp_test_csv.loc[(hp_test_csv.MasVnrArea > 1) & (hp_test_csv.MasVnrType == 'None'), 'MasVnrType'] = masvnrtype_mode[0]

In [13]:
hp_test_csv[(hp_test_csv['MasVnrType']!='None') & (hp_test_csv['MasVnrArea'].isna())]['MasVnrArea']=0
mvt_list = set(list(hp_test_csv[(hp_test_csv['MasVnrType']!='None') & (hp_test_csv['MasVnrArea']==0)]['MasVnrType']))

fill_masvnrtype(hp_test_csv, mvt_list)

[0]
176.0


In [14]:
hp_test_csv[hp_test_csv['MasVnrArea'].isna()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition


In [15]:
# adds the column 'GarageYrModified' and sets it == to 'YearRemodAdd' or 'GarageYrBlt'

# https://www.redfin.com/news/does-a-garage-add-value-to-a-home/

# according to the above, having a garage does not seem to impact houses in the midwest (besides Chicago)

hp_test_csv['GarageYrModified'] = np.where(hp_test_csv['YearRemodAdd']>hp_test_csv['YearBuilt'], hp_test_csv['YearRemodAdd'], hp_test_csv['GarageYrBlt'])

hp_test_csv['GarageYrModified'] = np.where(hp_test_csv['GarageYrModified']==0, hp_test_csv['YearRemodAdd'], hp_test_csv['YearBuilt'])
hp_test_csv['GarageYrModified'] = pd.to_numeric(hp_test_csv['GarageYrModified'])

In [16]:
should_not_have_nan_obj = ['MasVnrType', 'Electrical']
should_not_have_nan_num = ['LotFrontage', 'MasVnrArea']

obj_DNE_list = [i for i in cols_w_nan_obj if i not in should_not_have_nan_obj]
num_DNE_list = [i for i in cols_w_nan_num if i not in should_not_have_nan_num]

nan_col_obj_list_fillna(hp_test_csv, obj_DNE_list)
nan_col_obj_list_fillna(hp_test_csv, num_DNE_list)

In [17]:
cols_w_nan_obj2 = []

nan_col_list(hp_test_csv, cols_w_nan_obj, cols_w_nan_obj2)

[]

In [18]:
hp_test_csv['MasVnrType'].value_counts()

None       892
BrkFace    436
Stone      121
BrkCmn      10
Name: MasVnrType, dtype: int64

In [19]:
cols_w_nan_num2 = []

nan_col_list(hp_test_csv, cols_w_nan_num, cols_w_nan_num2)

['LotFrontage']

# Imputing Lot Frontage

In [26]:
cols_obj_dum = []
cols_num_dum = []

col_obj_list(hp_test_csv, hp_test_csv.columns, cols_obj_dum, cols_num_dum)
print(cols_obj_dum)
print('\n')
print(cols_num_dum)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',

In [27]:
hp_obj = hp_test_csv[cols_obj_dum]
hp_obj = pd.concat([hp_obj, hp_test_csv[['MSSubClass']]], axis=1)
hp_obj

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition,MSSubClass
0,RH,Pave,DNE,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,CBlock,TA,TA,No,Rec,LwQ,GasA,TA,Y,SBrkr,TA,Typ,DNE,Attchd,Unf,TA,TA,Y,DNE,MnPrv,DNE,WD,Normal,20
1,RL,Pave,DNE,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,TA,Y,SBrkr,Gd,Typ,DNE,Attchd,Unf,TA,TA,Y,DNE,DNE,Gar2,WD,Normal,20
2,RL,Pave,DNE,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Gd,Y,SBrkr,TA,Typ,TA,Attchd,Fin,TA,TA,Y,DNE,MnPrv,DNE,WD,Normal,60
3,RL,Pave,DNE,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,TA,TA,PConc,TA,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,Fin,TA,TA,Y,DNE,DNE,DNE,WD,Normal,60
4,RL,Pave,DNE,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,Gable,CompShg,HdBoard,HdBoard,None,Gd,TA,PConc,Gd,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,DNE,Attchd,RFn,TA,TA,Y,DNE,DNE,DNE,WD,Normal,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,Pave,DNE,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,Gable,CompShg,CemntBd,CmentBd,None,TA,TA,CBlock,TA,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,TA,Typ,DNE,DNE,DNE,DNE,DNE,Y,DNE,DNE,DNE,WD,Normal,160
1455,RM,Pave,DNE,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,Gable,CompShg,CemntBd,CmentBd,None,TA,TA,CBlock,TA,TA,No,Rec,Unf,GasA,TA,Y,SBrkr,TA,Typ,DNE,CarPort,Unf,TA,TA,Y,DNE,DNE,DNE,WD,Abnorml,160
1456,RL,Pave,DNE,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Detchd,Unf,TA,TA,Y,DNE,DNE,DNE,WD,Abnorml,20
1457,RL,Pave,DNE,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,Gable,CompShg,HdBoard,Wd Shng,None,TA,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,TA,Y,SBrkr,TA,Typ,DNE,DNE,DNE,DNE,DNE,Y,DNE,MnPrv,Shed,WD,Normal,85


In [28]:
hp_obj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 44 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1459 non-null   object
 1   Street         1459 non-null   object
 2   Alley          1459 non-null   object
 3   LotShape       1459 non-null   object
 4   LandContour    1459 non-null   object
 5   Utilities      1459 non-null   object
 6   LotConfig      1459 non-null   object
 7   LandSlope      1459 non-null   object
 8   Neighborhood   1459 non-null   object
 9   Condition1     1459 non-null   object
 10  Condition2     1459 non-null   object
 11  BldgType       1459 non-null   object
 12  HouseStyle     1459 non-null   object
 13  RoofStyle      1459 non-null   object
 14  RoofMatl       1459 non-null   object
 15  Exterior1st    1459 non-null   object
 16  Exterior2nd    1459 non-null   object
 17  MasVnrType     1459 non-null   object
 18  ExterQual      1459 non-null

# For Tree Models

In [29]:
hp_tree_noid = hp_test_csv.copy()
hp_tree = hp_tree_noid.drop(['Id'], axis=1)

hp_tree.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   MSSubClass        1459 non-null   int64  
 1   MSZoning          1459 non-null   object 
 2   LotFrontage       1232 non-null   float64
 3   LotArea           1459 non-null   int64  
 4   Street            1459 non-null   object 
 5   Alley             1459 non-null   object 
 6   LotShape          1459 non-null   object 
 7   LandContour       1459 non-null   object 
 8   Utilities         1459 non-null   object 
 9   LotConfig         1459 non-null   object 
 10  LandSlope         1459 non-null   object 
 11  Neighborhood      1459 non-null   object 
 12  Condition1        1459 non-null   object 
 13  Condition2        1459 non-null   object 
 14  BldgType          1459 non-null   object 
 15  HouseStyle        1459 non-null   object 
 16  OverallQual       1459 non-null   int64  


In [31]:
for i, x in enumerate(cols_obj_dum):
    hp_tree[x] = hp_tree[x].map(conversion[i])
    
hp_tree_nalist = hp_tree.columns[hp_tree.isna().any()].tolist()

for name in hp_tree_nalist:
    print(name)
    print(hp_tree[name].unique())
    print('\n')
    print('#'*50)
    print('\n')

MSZoning
[ 5.  6.  8.  3. nan]


##################################################


LotFrontage
[ 80.  81.  74.  78.  43.  75.  nan  63.  85.  70.  26.  21.  24. 102.
  94.  90.  79. 110. 105.  41. 100.  67.  60.  73.  92.  84.  39.  88.
  25.  30.  57.  68.  98. 120.  87. 119.  65.  56.  69.  50.  53.  52.
  51.  72.  86. 124.  44.  83.  64.  82.  38.  89.  35.  58.  66.  93.
  31.  76.  28.  61.  95. 129.  59.  77.  96.  47.  34. 117.  48.  62.
  42. 106. 112.  32. 115.  71.  45. 109. 113. 125. 101. 104. 108. 130.
 135.  36.  55. 136.  97.  91.  37.  22. 103.  99.  40. 123.  54. 107.
 150. 160. 195. 128.  33. 118. 134. 155. 126.  46. 149. 200. 121. 131.
 114.  49. 133. 140.]


##################################################


Utilities
[ 1. nan]


##################################################


BldgType
[ 1.  4. nan]


##################################################


Exterior1st
[15. 16.  7. 11.  9.  6. 17.  4.  1.  3. 14.  2. nan  5.]


################################

In [ ]:
hp.loc[hp['MSZoning']=='C (all)', 'MSZoning'] = 'C'

hp.loc[hp['BldgType']=='2fmCon', 'BldgType'] = '2FmCon'
hp.loc[hp['BldgType']=='Duplex', 'BldgType'] = 'Duplx'
hp.loc[hp['BldgType']=='Twnhs', 'BldgType'] = 'TwnhsI'

hp.loc[hp['Exterior2nd']=='Wd Shng', 'Exterior2nd'] = 'WdShing'
hp.loc[hp['Exterior2nd']=='CmentBd', 'Exterior2nd'] = 'CemntBd'
hp.loc[hp['Exterior2nd']=='Brk Cmn', 'Exterior2nd'] = 'BrkComm'